# LUME-services

### Configuration:

Set the following environment variables:

```
export LUME_MODEL_DB__HOST=127.0.0.1
export LUME_MODEL_DB__USER=root
export LUME_MODEL_DB__PASSWORD=test
export LUME_MODEL_DB__PORT=3306
export LUME_MODEL_DB__DATABASE=model_db
export LUME_MODEL_DB__CONNECTION__POOL_SIZE=1
export LUME_PREFECT__SERVER__TAG=core-1.2.4
export LUME_PREFECT__SERVER__HOST_PORT=4200
export LUME_PREFECT__SERVER__HOST=http://localhost
export LUME_PREFECT__AGENT__HOST_PORT=5000
export LUME_PREFECT__AGENT__HOST=http://localhost
export LUME_RESULTS_DB__HOST=127.0.0.1
export LUME_RESULTS_DB__PORT=27017
export LUME_RESULTS_DB__USERNAME=root
export LUME_RESULTS_DB__PASSWORD=password
export LUME_RESULTS_DB__DATABASE=test
export LUME_PREFECT__BACKEND=server 
export LUME_BACKEND=docker
```

Start the docker services using:
```
lume-services docker start-services
```

In [ ]:
from lume_services.services.models.db import ModelDBConfig
from lume_services.services.results.mongodb import MongodbResultsDBConfig
from lume_services.services.scheduling.backends.server import (
    PrefectAgentConfig,
    PrefectConfig,
    PrefectServerConfig,
)

from lume_services.services.files.filesystems import (
    LocalFilesystem,
    MountedFilesystem,
)
from lume_services.config import LUMEServicesSettings


model_db_config = ModelDBConfig(
        host="127.0.0.1",
        port="3306",
        user="root",
        password="test",
        database="model_db",
)

results_db_config = MongodbResultsDBConfig(
    port="27017",
    host="127.0.0.1",
    username="root",
    database="test",
    password="password",
)

prefect_config = PrefectConfig(
    server=PrefectServerConfig(
        host="http://localhost", host_port="4200", tag="core-1.2.4"
    ),
    agent=PrefectAgentConfig(host="http://localhost", host_port="5000"),
    backend="server",
)


settings = LUMEServicesSettings(
    model_db=model_db_config,
    results_db=results_db_config,
    prefect=prefect_config,
    backend="docker",
    mounted_filesystem=None,
)

## Use LUME-services config to create injected services

In [ ]:
from lume_services import config
config.configure(settings)

# initialize singletons
model_db_service = config.context.model_db_service()
results_db_service = config.context.results_db_service()

## Store example model, deployment, flow, and project

In [ ]:
# model
author = "Jackie Garrahan"
laboratory = "slac"
facility = "lcls"
beampath = "cu_hxr"
description = "test_model"

# deployment
version = "v0.0"
sha256 = "placeholder"
asset_dir = None  # opt
source = "my source"
is_live = 1
image = "placeholder"

# project
project = "my_project"
project_description = "placeholder"

# flow
flow_id_placeholder = "test"
flow_name = "my_test_flow"

In [ ]:
model_id = model_db_service.store_model(
    author=author,
    laboratory=laboratory,
    facility=facility,
    beampath=beampath,
    description=description,
)

In [ ]:
deployment_id = model_db_service.store_deployment(
    model_id=model_id,
    version=version,
    asset_dir=asset_dir,
    sha256=sha256,
    source=source,
    is_live=is_live,
    image=image,
)


deployment = model_db_service.get_deployment(deployment_id=deployment_id)

In [ ]:
project_name = model_db_service.store_project(
    project_name=project, description=project_description
)


In [ ]:
flow_id = model_db_service.store_flow(
    project_name=project_name,
    deployment_id=deployment_id,
    flow_id=flow_id_placeholder,
    flow_name=flow_name,
)

In [ ]:
from lume_services.models import Model

my_model = Model(model_id=model_id, model_db_service=model_db_service)


In [ ]:
my_model

In [ ]:
my_model.load_deployment(model_db_service=model_db_service)